# 🦜🔗 LangChain React Agent Demo

In [ ]:
!pip install -r requirements.txt

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI
from dotenv import load_dotenv

load_dotenv()

Create a llm and load tools for search and math

In [ ]:
llm = OpenAI(temperature=0)

tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [ ]:
agent.run("Wie alt ist Dieter Bohlen? Wie alt ist seine Frau? Was ist die Altersdifferenz?")